In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# make fake data
np.random.seed(42)  # for reproducibility 
dates = pd.date_range('2024-01-01', '2024-01-07', freq='10min')  # week of data
df = pd.DataFrame({
    'date': dates,
    'transaction_id': range(len(dates)),
    'user_id': np.random.randint(1, 1000, len(dates)),
    'transaction_amount': np.random.uniform(10, 500, len(dates))
})

# show sample of raw data
print('Sample of random data:')
print(df.head())
print('\nDataset shape:', df.shape)
print('\nData overview:')
df.info()

Sample of random data:
                 date  transaction_id  user_id  transaction_amount
0 2024-01-01 00:00:00               0      103          493.140521
1 2024-01-01 00:10:00               1      436          379.155311
2 2024-01-01 00:20:00               2      861          194.367197
3 2024-01-01 00:30:00               3      271           50.915351
4 2024-01-01 00:40:00               4      107          390.801989

Dataset shape: (865, 4)

Data overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                865 non-null    datetime64[ns]
 1   transaction_id      865 non-null    int64         
 2   user_id             865 non-null    int32         
 3   transaction_amount  865 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1)
memory usage: 23.8 KB


In [2]:
# prep datetime stuff
df['hour'] = df['date'].dt.floor('h')  # round to hour

# get hourly stats
stats = df.groupby('hour').agg({
    'transaction_id': 'nunique',  # unique txns
    'user_id': 'nunique',         # unique users
    'transaction_amount': 'sum'    # total $
}).reset_index()

print('Hourly stats sample:')
print(stats.head())
print('\nStats summary:')
print(stats.describe())

Hourly stats sample:
                 hour  transaction_id  user_id  transaction_amount
0 2024-01-01 00:00:00               6        6         1791.998451
1 2024-01-01 01:00:00               6        6         1243.062501
2 2024-01-01 02:00:00               6        6         1172.922979
3 2024-01-01 03:00:00               6        6         1862.304633
4 2024-01-01 04:00:00               6        6         1744.131094

Stats summary:
                      hour  transaction_id     user_id  transaction_amount
count                  145      145.000000  145.000000          145.000000
mean   2024-01-04 00:00:00        5.965517    5.958621         1513.854150
min    2024-01-01 00:00:00        1.000000    1.000000           49.256275
25%    2024-01-02 12:00:00        6.000000    6.000000         1290.180379
50%    2024-01-04 00:00:00        6.000000    6.000000         1554.227941
75%    2024-01-05 12:00:00        6.000000    6.000000         1782.293333
max    2024-01-07 00:00:00        6.

In [3]:
# find peak hour each day
peaks = df.groupby(['date', 'hour'])['transaction_amount'].sum().reset_index()
best_hours = peaks.loc[peaks.groupby('date')['transaction_amount'].idxmax()]

print('Peak hours by day:')
print(best_hours)

# quick stats about peaks
print('\nPeak hour stats:')
print(f"Average peak hour amount: ${best_hours['transaction_amount'].mean():.2f}")
print(f"Most common peak hour: {best_hours['hour'].dt.hour.mode().iloc[0]}:00")

Peak hours by day:
                   date                hour  transaction_amount
0   2024-01-01 00:00:00 2024-01-01 00:00:00          493.140521
1   2024-01-01 00:10:00 2024-01-01 00:00:00          379.155311
2   2024-01-01 00:20:00 2024-01-01 00:00:00          194.367197
3   2024-01-01 00:30:00 2024-01-01 00:00:00           50.915351
4   2024-01-01 00:40:00 2024-01-01 00:00:00          390.801989
..                  ...                 ...                 ...
860 2024-01-06 23:20:00 2024-01-06 23:00:00          359.902597
861 2024-01-06 23:30:00 2024-01-06 23:00:00          402.846761
862 2024-01-06 23:40:00 2024-01-06 23:00:00          176.330594
863 2024-01-06 23:50:00 2024-01-06 23:00:00          409.264306
864 2024-01-07 00:00:00 2024-01-07 00:00:00           49.256275

[865 rows x 3 columns]

Peak hour stats:
Average peak hour amount: $253.77
Most common peak hour: 0:00
